In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# load data; current representation pandas dataframe
file = "IMDB-Movie-Data.csv"
data = pd.read_csv(file)

def data_preprocessing(data):
    # lower case everything; more preprocessing can be done 
    for column in ['Title', 'Genre', 'Description', 'Director', 'Actors']:
        data[column] = data[column].apply(lambda x: x.lower())
    data['Genre'] = data['Genre'].apply(lambda x: x.replace(',', ' '))
    return data

data = data_preprocessing(data)
data.sample(2)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
171,172,kubo and the two strings,animation adventure family,a young boy named kubo must locate a magical s...,travis knight,"charlize theron, art parkinson, matthew mccona...",2016,101,7.9,72778,48.02,84.0
424,425,london has fallen,action crime drama,"in london for the prime minister's funeral, mi...",babak najafi,"gerard butler, aaron eckhart, morgan freeman,a...",2016,99,5.9,100702,62.40,28.0


In [6]:
# get query
query = input("Enter Movie query: ")
query = list(query.split())
print("Query ", query)

Enter Movie query: clown
Query  ['clown']


In [7]:
# scoring function
def get_score(row, query):
    score = 0
    for word in query:
        for col in ['Title', 'Genre', 'Description', 'Director', 'Actors', 'Year']:
            if word in str(row[col]):
                # for now matching a keyword in any column counts the same (insufficient for now ofc)
                score += 1    
    return score


def rank_movies(data, query):
    data['Score'] = 0
    for i, row in tqdm(data.iterrows()):
        score = get_score(row, query)
        data.at[i,'Score'] = score

    # first sort on Score then Rating
    data.sort_values(['Score', 'Rating'], ascending = False, inplace = True)
    
    return data
    
ranked_data = rank_movies(data, query)

# display top 5 movies
ranked_data.head(5)

1000it [00:00, 6892.43it/s]


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Score
361,362,clown,horror thriller,a loving father finds a clown suit for his son...,jon watts,"andy powers, laura allen, peter stormare, chri...",2014,100,5.7,14248,0.05,42.0,2
776,777,31,horror thriller,five carnival workers are kidnapped and held h...,rob zombie,"malcolm mcdowell, richard brake, jeff daniel p...",2016,102,5.1,10871,0.78,35.0,1
54,55,the dark knight,action crime drama,when the menace known as the joker wreaks havo...,christopher nolan,"christian bale, heath ledger, aaron eckhart,mi...",2008,152,9.0,1791916,533.32,82.0,0
80,81,inception,action adventure sci-fi,"a thief, who steals corporate secrets through ...",christopher nolan,"leonardo dicaprio, joseph gordon-levitt, ellen...",2010,148,8.8,1583625,292.57,74.0,0
117,118,dangal,action biography drama,former wrestler mahavir singh phogat and his t...,nitesh tiwari,"aamir khan, sakshi tanwar, fatima sana shaikh,...",2016,161,8.8,48969,11.15,NaN,0


In [5]:
# display results
